If I want to write a step, it should be as easy as writing a function

In [2]:
def my_step(context, arg1, arg2, arg3=None, arg4=4):
    ...

So, a step is just a function that takes a `context` as its first argument, then some number of positional and keyword arguments.

But, that raises a question, how do you compose these steps together?

In [3]:
@script
def made_of_steps():
    my_step(1,2, arg3=3)
    another_step()

NameError: name 'script' is not defined

I might even have a test script with some prefixing for clarity:

In [ ]:
@scenario
def test_some_things():
    Given.some_precondition()
    When.something_happens()
    Then.some_condition_is_satisfied()

It would all just get rewritten to

In [4]:
def test_some_things_rewritten(context):
    some_precondition(context)
    something_happens(context)
    some_condition_is_satisfied(context)

It would be nice if you could annotate a step function so that it could be introspected for accesses to the context. The purpose would be to determine the items that are accessed from context.

In [9]:
@step
def my_step(context, arg1, arg2):
    arg3 = context.get("arg3")
    arg4 = context.get("arg4")
    

NameError: name 'step' is not defined

You could go a bit further, too, and use constants to indicate desired item, for better saftey. What's more, you could use a constant that also includes type information to further clarify the contextual dependency

In [10]:
from steps.http import LAST_RESPONSE
from requests import Response

@step
def my_step(context, arg1, arg2):
    response = context.get(LAST_RESPONSE, Response)

ModuleNotFoundError: No module named 'steps'

This is roughly equivalent to:

In [11]:
def my_step(context, arg1, arg2, response: Response):
    ...
    
my_step(context, arg1, arg2, context.get(LAST_RESPONSE, Response))

NameError: name 'Response' is not defined

But, with a bit more machinery around it

Now... what could we do with Type annotations?

In [12]:
@step
def my_step(context, arg1: int, arg2: intID):
    response = context.LAST_RESPONSE[Response]
    ...
    
@step
def other_step(context, arg1: int, arg2: intID):
    response = context.LAST_RESPONSE[OtherTypeOfResponse]
    ...
    

NameError: name 'step' is not defined

Interestingly... I'd say that steps are a map: `Context -> Context`. The return of `Context` is implicit...